# Notebook for XSEDE proposal

### Resource usage.

In [48]:
from yt import *
import numpy as np
from __future__ import print_function # stupid 2.7 print was goofing up.

Lets define a few global things to clean up the notebook a bit...

In [49]:
pc = 3.086e18
AU = 1.5e13
hr = 60*60
yr = hr*24.*365.25
Myr = 3.15e13

# Radiation on or off for this run?
rad_on = True
# Winds on or off for this run?
winds_on = True

# Size of the zoomed in region
inner_box = 50*pc

# Target refinement
dx = 1500*AU

# Number of cells in a block in 1-d
nx = 16
# Target number of blocks per proc. ~200 gives good performance.
blksPerProc = 200.

# Number of refinements to go from inner_box to dx
num_zoom_lvls = int(np.ceil(np.log10(50*pc/(nx*dx))/np.log10(2)))

# Now find the actual dx for this many levels of refinement

dx = inner_box/(nx*2**(num_zoom_lvls))

print("For a zoomed region", inner_box/pc, "pc in size it takes", num_zoom_lvls,
       "refinements to hit target refinement", dx/AU, "AU")

For a zoomed region 50.0 pc in size it takes 9 refinements to hit target refinement 1255.69661458 AU


Number of cells in the simulation.
Assume the zoom in leaves us 384 blocks in the inner zoomed region for ~50 pc^3 area,
which is what we get for the Bullhead cloud.

After that the volume filling fraction is about 1/8, so the number of blocks generally doubles per level of refinement. Here we use 2.5 to be safe (which was what Olson put in the original NSF proposal).  [***factor = 2.0 in the notebook, though***]

Also assume blocks have 16^3 cells in each.

In [50]:
factor = 2.0

# Number of blocks for 50 pc zoom in region, based on the bull head cloud.
# These are all the background blocks that *don't* include the zoomed region.
numBlksOuter=3500

# Now the zoomed region at 5 levels of refinement and over a 50 pc region square
# should cover about 4^3=64 blocks. Then from there each level should just more than double the
# last level. But for some reason the first level in (lef=6) has 364 blocks, not
# 64*2.5=160 blocks. After that though it follows the 2.5 rule pretty well.

numBlksZoom1=384

numBlksLevel = numBlksZoom1

numBlksZoomTotal = numBlksZoom1

# Number of blocks if we assume a cloud that is an order magnitude larger has twice the
# radius and therefore 2^3 times more blocks. This is likely conservative, since the
# cloud is probably more dense and not as large.
numBlksBig=numBlksZoom1*(2^3)


# Each level has factor*numBlksPreviousLevel number of blocks.
# Here we sum up those blocks to get the total. Note we start
# with only those blocks in the region we zoomed in on!
# Previously we mistakingly calculated it with all the blocks
# at the previous level.

# Note here we started at lzoom + 1 so no need to add 1 anymore.

for i in range(1, num_zoom_lvls-1):
    
    print(i)
    
    numBlksLevel = factor*numBlksLevel
     
    numBlksZoomTotal = numBlksZoomTotal + numBlksLevel
    
    print(numBlksLevel)
    
numBlksTotal = numBlksZoomTotal + numBlksOuter
    
print(numBlksTotal, "blocks total.")

print(numBlksTotal/blksPerProc)

numCells =  numBlksTotal*(16**3)

print(numBlksTotal/blksPerProc)

print(numCells, "cells in the simulation after fragmentation.")

1
768.0
2
1536.0
3
3072.0
4
6144.0
5
12288.0
6
24576.0
7
49152.0
101420.0 blocks total.
507.1
507.1
415416320.0 cells in the simulation after fragmentation.


Here we calculate the average timestep assuming that the highest temperature is due to the radaition feedback. We assume it takes 2 Myr for the gas to collapse to the Jeans criterion at the highest level of refinement and star formation to start.

In [51]:
# Temp for no winds (just radiation)
lowT = 1e4 * units.K   #corrected from 10e4 = 1e5  M-MML 22:15 14 Oct 16
# Temp for with winds.
highT = 1e6 * units.K  #               10e6 = 1e7

# Atomic weight for ionized hydrogren and H2.
mu_i = 0.62
mu_n = 2.4

# Sound speed.
csLow = np.sqrt(5./3. *units.boltzmann_constant_cgs.v*lowT.v /(mu_n* units.mass_hydrogen_cgs.v))

if winds_on:
    csHigh = np.sqrt(5./3. *units.boltzmann_constant_cgs.v*highT.v /(mu_i* units.mass_hydrogen_cgs.v))
else:
    csHigh=csLow
    
print (csLow/1.0e5, csHigh/1.0e5)  #km/s

# 500 AU cell size at highest refinement.
#dx = 500 * units.AU.in_cgs().v

print(dx, "is cell width in cm")

# Timestep.
lowTS = dx/csLow
highTS = dx/csHigh

print(lowTS/(60*60*24*365.25), "years for low temp timesteps")
print(highTS/(60*60*24*365.25), "years for high temp timesteps")


# Number of steps for a 10 Myr simulation.
numSteps = 2*Myr/lowTS + 8*Myr/highTS # Radiation is on.

print(numSteps, "steps in the simulation")



7.56862496372 148.910993777
1.88354492188e+16 is cell width in cm
788.596834592 years for low temp timesteps
40.0816188061 years for high temp timesteps
201759.954664 steps in the simulation


Now we calculate the amount of computer time needed to do the simulation. Here we assume:

1) An average Flash MHD update takes ~150 microseconds (Olson's timing). 

2) An average radiation takes about the same amount of time as an MHD update (Wise & Abel 2011, Stone @ PiTP 2016)

In [52]:
if rad_on:
    timePerStep = 300e-6 # If Radiation is on
else:
    timePerStep = 150e-6 # If Radiation is off


print("{:.2e}".format(numSteps*numCells*timePerStep/hr), "hrs is total CPU time")

6.98e+06 hrs is total CPU time


### Storage requirements

Assume that we make a plot file every 10,000 years

In [53]:
pltFreq = 1e5
runTime = 1e7
numPlots = runTime/pltFreq

numHyVars = 30
numCells = numCells
numPartVars = 15
numParts = 1e4



sizeOfVar = 64
sizeOfGB = 2.**30

tempStorage = numPlots*(numHyVars*numCells+numPartVars*numParts)*sizeOfVar/sizeOfGB

print(tempStorage)

74283.1206322


Lets turn it all into a function that can return the timing.

In [96]:
def code_run(inner_box, dx, rad_on=True, winds_on=True):

    print("Checking run time with rad_on=", rad_on, "and winds_on=", winds_on)
    # Number of refinements to go from inner_box to dx
    num_zoom_lvls = int(np.ceil(np.log10(50*pc/(nx*dx))/np.log10(2)))

    # Now find the actual dx for this many levels of refinement

    dx = inner_box/(nx*2**(num_zoom_lvls))

    print("For a zoomed region", inner_box/pc, "pc in size it takes", num_zoom_lvls,
           "refinements to hit target refinement", dx/AU, "AU")

    factor = 2.0

    # Number of blocks for 50 pc zoom in region, based on the bull head cloud.
    # These are all the background blocks that *don't* include the zoomed region.
    numBlksOuter=3500

    # Now the zoomed region at 5 levels of refinement and over a 50 pc region square
    # should cover about 4^3=64 blocks. Then from there each level should just more than double the
    # last level. But for some reason the first level in (lef=6) has 364 blocks, not
    # 64*2.5=160 blocks. After that though it follows the 2.5 rule pretty well.

    numBlksZoom1=384

    numBlksLevel = numBlksZoom1

    numBlksZoomTotal = numBlksZoom1

    # Number of blocks if we assume a cloud that is an order magnitude larger has twice the
    # radius and therefore 2^3 times more blocks. This is likely conservative, since the
    # cloud is probably more dense and not as large.
    numBlksBig=numBlksZoom1*(2^3)


    # Each level has factor*numBlksPreviousLevel number of blocks.
    # Here we sum up those blocks to get the total. Note we start
    # with only those blocks in the region we zoomed in on!
    # Previously we mistakingly calculated it with all the blocks
    # at the previous level.

    # Note here we started at lzoom + 1 so no need to add 1 anymore.

    for i in range(1, num_zoom_lvls-1):

        #print(i)

        numBlksLevel = factor*numBlksLevel

        numBlksZoomTotal = numBlksZoomTotal + numBlksLevel

        #print(numBlksLevel)

    numBlksTotal = numBlksZoomTotal + numBlksOuter

    print(numBlksTotal, "blocks total.")

    numCells =  numBlksTotal*(16**3)

    #print(numBlksTotal/blksPerProc, "number of processors needed.")

    print(numCells, "cells in the simulation after fragmentation.")
    
    # Temp for no winds (just radiation)
    lowT = 1e4 * units.K   #corrected from 10e4 = 1e5  M-MML 22:15 14 Oct 16
    # Temp for with winds.
    highT = 1e6 * units.K  #               10e6 = 1e7

    # Atomic weight for ionized hydrogren and H2.
    mu_i = 0.62
    mu_n = 2.4

    # Sound speed.
    csLow = np.sqrt(5./3. *units.boltzmann_constant_cgs.v*lowT.v /(mu_n* units.mass_hydrogen_cgs.v))

    if winds_on:
        csHigh = np.sqrt(5./3. *units.boltzmann_constant_cgs.v*highT.v /(mu_i* units.mass_hydrogen_cgs.v))
    else:
        csHigh=csLow

    print (csLow/1.0e5, "km/s = csLow", csHigh/1.0e5, "km/s =csHigh")  #km/s

    # 500 AU cell size at highest refinement.
    #dx = 500 * units.AU.in_cgs().v

    print(dx/AU, "is cell width in AU")

    # Timestep.
    lowTS = dx/csLow
    highTS = dx/csHigh

    print(lowTS/(60*60*24*365.25), "years for low temp timesteps")
    print(highTS/(60*60*24*365.25), "years for high temp timesteps")
    
    print ("Courant # for highTS=", csHigh*highTS/dx)


    # Number of steps for a 10 Myr simulation.
    numSteps = 2*Myr/lowTS + 8*Myr/highTS # Radiation is on.

    print(numSteps, "steps in the simulation")

    if rad_on:
        timePerStep = 300e-6 # If Radiation is on
    else:
        timePerStep = 150e-6 # If Radiation is off

    totalCPUtime = numSteps*numCells*timePerStep/hr
    print("{:.2e}".format(totalCPUtime), "hrs is total CPU time")
    
    cs2Cold = (5./3. * 1.3e-16 * 10 / (2.4*1.67e-24))
    
    density_at_12dx = (np.pi*cs2Cold / 
                       (6.67e-8*(12.0*dx)**2.0)
                       /1.67e-24)
    
    print("Number density at MHD Jeans Length (12 dx) =",
          "{:.2e}".format(density_at_12dx), "cm^-3")
    
    return totalCPUtime, dx

In [97]:
both_on=code_run(inner_box, dx)[0]

Checking run time with rad_on= True and winds_on= True
For a zoomed region 50.0 pc in size it takes 9 refinements to hit target refinement 1255.69661458 AU
101420.0 blocks total.
415416320.0 cells in the simulation after fragmentation.
7.56862496372 km/s = csLow 148.910993777 km/s =csHigh
1255.69661458 is cell width in AU
788.596834592 years for low temp timesteps
40.0816188061 years for high temp timesteps
Courant # for highTS= 1.0
201759.954664 steps in the simulation
6.98e+06 hrs is total CPU time
Number density at MHD Jeans Length (12 dx) = 2.98e+05 cm^-3


In [98]:
winds_only=code_run(inner_box, dx, rad_on=False, winds_on=True)[0]

Checking run time with rad_on= False and winds_on= True
For a zoomed region 50.0 pc in size it takes 9 refinements to hit target refinement 1255.69661458 AU
101420.0 blocks total.
415416320.0 cells in the simulation after fragmentation.
7.56862496372 km/s = csLow 148.910993777 km/s =csHigh
1255.69661458 is cell width in AU
788.596834592 years for low temp timesteps
40.0816188061 years for high temp timesteps
Courant # for highTS= 1.0
201759.954664 steps in the simulation
3.49e+06 hrs is total CPU time
Number density at MHD Jeans Length (12 dx) = 2.98e+05 cm^-3


In [99]:
rad_only=code_run(inner_box, dx, winds_on=False)[0]

Checking run time with rad_on= True and winds_on= False
For a zoomed region 50.0 pc in size it takes 9 refinements to hit target refinement 1255.69661458 AU
101420.0 blocks total.
415416320.0 cells in the simulation after fragmentation.
7.56862496372 km/s = csLow 7.56862496372 km/s =csHigh
1255.69661458 is cell width in AU
788.596834592 years for low temp timesteps
788.596834592 years for high temp timesteps
Courant # for highTS= 1.0
12657.6055388 steps in the simulation
4.38e+05 hrs is total CPU time
Number density at MHD Jeans Length (12 dx) = 2.98e+05 cm^-3


In [100]:
both_off=code_run(inner_box, dx, rad_on=False, winds_on=False)[0]

Checking run time with rad_on= False and winds_on= False
For a zoomed region 50.0 pc in size it takes 9 refinements to hit target refinement 1255.69661458 AU
101420.0 blocks total.
415416320.0 cells in the simulation after fragmentation.
7.56862496372 km/s = csLow 7.56862496372 km/s =csHigh
1255.69661458 is cell width in AU
788.596834592 years for low temp timesteps
788.596834592 years for high temp timesteps
Courant # for highTS= 1.0
12657.6055388 steps in the simulation
2.19e+05 hrs is total CPU time
Number density at MHD Jeans Length (12 dx) = 2.98e+05 cm^-3


In [101]:
total_time=both_on+both_off+winds_only+rad_only

print("Total time in CPU hours =", "{:.2e}".format(total_time))

Total time in CPU hours = 1.11e+07
